# WechatSogou

### algorithm:
1. import module
2. get gzh historical articles
3. save the html to local disk
4. insert into database or update sids

## 1. import modules

In [96]:
import wechatsogou
import random
import requests
import datetime
import time

In [44]:
# 直连
ws_api = wechatsogou.WechatSogouAPI()

# 验证码输入错误的重试次数，默认为1
ws_api = wechatsogou.WechatSogouAPI(captcha_break_time=3)

# 所有requests库的参数都能在这用
# 如 配置代理，代理列表中至少需包含1个 HTTPS 协议的代理, 并确保代理可用
ws_api = wechatsogou.WechatSogouAPI(proxies={
    "http": "127.0.0.1:8888",
    "https": "127.0.0.1:8888",
})

# 如 设置超时
ws_api = wechatsogou.WechatSogouAPI(timeout=1)


In [ ]:
ws_api.get_gzh_info('南航青年志愿者')

In [21]:
gzh = ws_api.search_gzh('付鹏的财经世界')

In [22]:
list(gzh)

[{'open_id': 'oIWsFt94fGZWgq7sP6RyKLNfWV_c',
  'profile_url': 'http://mp.weixin.qq.com/profile?src=3&timestamp=1561689870&ver=1&signature=91L84vGPkcdYvAE9nRQAdU-xqHO0xxgQICTT42IiLQw7FGJFzQe8ER*XXNCRectTqr-m9kp4Hs-vCYB1sPK99Q==',
  'headimage': 'https://img01.sogoucdn.com/app/a/100520090/oIWsFt94fGZWgq7sP6RyKLNfWV_c',
  'wechat_name': '付鹏的财经世界',
  'wechat_id': 'GlobalMacroHedge',
  'qrcode': 'https://img01.sogoucdn.com/v2/thumb?t=2&url=http%3A%2F%2Fmp.weixin.qq.com%2Frr%3Fsrc%3D3%26timestamp%3D1561689870%26ver%3D1%26signature%3DoVfpThM3K3Kk-PNeJiYRuSky*Q-G-8fYqU2PvQNEbjiHo1*B75a3WwqDKEr1W-SfKFhkISzgkiu*I2ma7IDZFrOFiUiO0nSO7YiC7fM3Cf8%3D&appid=200580',
  'introduction': '「金融是件专业的事情,投资不是拍脑袋决定的」 - 记录是一种维持连续思考最好的方式,我是付鹏,这是我眼里的财经世界',
  'authentication': '\n',
  'post_perm': 5,
  'view_perm': 6906}]

In [41]:
articles = ws_api.search_article('付鹏的财经世界')

In [42]:
list(articles)

please input code: dx8j7b


[{'article': {'title': '窘迫的央行-背后越来越庞大的负利率债券规模',
   'url': '',
   'imgs': ['https://img01.sogoucdn.com/net/a/04/link?appid=100520033&url=http://mmbiz.qpic.cn/mmbiz_jpg/lfVqLXNGC3b8nHgzsSpMOMbHGibjQkIhUYcVxv5iah3tFPI7ib8ylhY9rPK7tDnUCkYyJjO8SMXibXrTjTP09qdjAw/0?wx_fmt=jpeg'],
   'abstract': '恰恰并不是ECB的进一步政策,而是在数月后开始的欧洲政坛的风云跌宕;(完)付鹏的财经世界GlobalMacroHedge',
   'time': 1561139219},
  'gzh': {'profile_url': 'http://mp.weixin.qq.com/profile?src=3&timestamp=1561709391&ver=1&signature=91L84vGPkcdYvAE9nRQAdU-xqHO0xxgQICTT42IiLQw7FGJFzQe8ER*XXNCRectTYOp00n0q5-6-PDwzr991Zw==',
   'headimage': 'http://wx.qlogo.cn/mmhead/Q3auHgzwzM6ScGzFne2oGzSHicjDW2DrUjCSPzfDGtt8ibARAWG230aA/0',
   'wechat_name': '付鹏的财经世界',
   'isv': 1}},
 {'article': {'title': '黄金的温故而知新日记:对比2016年',
   'url': 'http://mp.weixin.qq.com/s?src=11&timestamp=1561709391&ver=1695&signature=Rxj4z0eanqTsJBQoFsgVzfX-aeB7pZRRr3wxrX5x2K7QwPSLy9pQ*R*ARsEo-kgdU5WtzLw3kBAq-2qdz4jPa9ktjTyJID-ZUnf55DrJLqYFQH8EtI*q86AMp*TuuT2W&new=1',
   'imgs': ['htt

In [33]:
ws_api.get_sugg('财经')

['财经你知道',
 '财经多看点',
 '财经扩音器',
 '财经法律顾问',
 '财经浩望角周刊',
 '财经热门快讯',
 '财经琳界点',
 '财经百晓童',
 '财经百讯',
 '财经见闻']

In [106]:
history = ws_api.get_gzh_article_by_history('长赢指数投资')

In [107]:
list(history)

['gzh', 'article']

In [108]:
history

{'gzh': {'wechat_name': '长赢指数投资',
  'wechat_id': 'chinaetfs',
  'introduction': '指数化投资适合绝大多数人。利用市场估值与投资者情绪的变化，对指数长期持有，低买高卖，是普通投资者在资本市场获得财富的最佳方式。本公众号定期给出投资指数ETF的建议，过往投资成绩出色，值得你拥有。',
  'authentication': '指数化投资适合绝大多数人。利用市场估值与投资者情绪的变化，对指数长期持有，低买高卖，是普通投资者在资本市场获得财富的最佳方式。本公众号定期给出投资指数ETF的建议，过往投资成绩出色，值得你拥有。',
  'headimage': 'http://wx.qlogo.cn/mmhead/Q3auHgzwzM7onibdZLKeNBoHFp5j7Ykib009RmxZaS18icYlh3QNZXRMA/0'},
 'article': [{'send_id': 1000000133,
   'datetime': 1560820311,
   'type': '49',
   'main': 1,
   'title': '2019年6月ETF计划（二）：买入两份',
   'abstract': '本次交易买入2份',
   'fileid': 505925561,
   'content_url': 'http://mp.weixin.qq.com/s?timestamp=1561716796&src=3&ver=1&signature=O2pYvv46qlWarTl*ulaKNxm4QWILfwF1S-p3QtD4C9g*DEqgFFWuqgGjZrrq9yMi1OJBegtcUDVwNTwmJHNleoKGZN1F3dyRwdP3cwhdwGWkEJRNyXa-BLHlOGE6Zm-4bZNGC322YVnHfpysGfhykA6FwrBhskxbiPfjWBFBX20=',
   'source_url': '',
   'cover': 'http://mmbiz.qpic.cn/mmbiz_jpg/SEPick5M9xjM1mtDXT8ychzEtlfSqAH9YsLdTK0Bic78PSvG1rgFCqKljSnqwdtvacAVick4CHkoAFianXu

In [115]:
for ci in history['article']:
    uachoice = random.choice(ua)
    headers = {'Cookie': """SUID=2F2E40613921940A000000005C7C8A68; SUV=0054C7AB61402E2F5C7C8A6C4E9DE485; IPLOC=CN3100; UM_distinctid=16af8a3d10b1c6-00075670c610bf-e353165-1fa400-16af8a3d10c66e; ABTEST=5|1560408316|v1; JSESSIONID=aaafTsCy_KK2le_X5lqRw; PHPSESSID=38sddn82q3fr3mihpq5s9j09j7; SNUID=A3091D2C494FC612B1C5C56E4A2CA7BE; sct=7""",
    "User-Agent": uachoice}
    author = ci['author']
    title = ci['title']
    dt = ci['datetime']
    dt = datetime.datetime.fromtimestamp(
        int(dt)
    ).strftime('%Y_%m_%d')
    url = ci['content_url']
    print(title)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        article = response.text
        fpath = '{0}_{1}_{2}.html'.format(author, dt, title)
        fpath = fpath.replace('/', '')
        print(fpath)
        with open(fpath, 'w') as fp:
            fp.write(response.text)
    time.sleep(2)

2019年6月ETF计划（二）：买入两份
ETF拯救世界_2019_06_18_2019年6月ETF计划（二）：买入两份.html
2019年6月ETF计划（一）：买入两份
ETF拯救世界_2019_05_30_2019年6月ETF计划（一）：买入两份.html
波段策略.网格之三：网格策略进阶/2.0版
ETF拯救世界_2019_05_26_波段策略.网格之三：网格策略进阶2.0版.html
2019年5月ETF计划（二）：买入两份；卖出一份
ETF拯救世界_2019_05_15_2019年5月ETF计划（二）：买入两份；卖出一份.html
波段策略.网格之二：网格策略基础/1.0版
ETF拯救世界_2019_05_06_波段策略.网格之二：网格策略基础1.0版.html
波段策略.网格之一：写在前面、体系以及策略
ETF拯救世界_2019_04_28_波段策略.网格之一：写在前面、体系以及策略.html
2019年5月ETF计划（一）：买入一份
ETF拯救世界_2019_04_26_2019年5月ETF计划（一）：买入一份.html
2019年4月ETF计划（一）：买入两份
ETF拯救世界_2019_03_28_2019年4月ETF计划（一）：买入两份.html
围炉夜话（5）：你的肩膀可以扛起多少信任？
ETF拯救世界_2019_03_06_围炉夜话（5）：你的肩膀可以扛起多少信任？.html
2019年3月ETF计划（一）：150不交易，S买入一份
ETF拯救世界_2019_02_28_2019年3月ETF计划（一）：150不交易，S买入一份.html


'http://mp.weixin.qq.com/s?timestamp=1561716796&src=3&ver=1&signature=O2pYvv46qlWarTl*ulaKNxm4QWILfwF1S-p3QtD4C9g*DEqgFFWuqgGjZrrq9yMi1OJBegtcUDVwNTwmJHNlevAP26OtvzYI9Skd4EEk7UFjipt4thoy3J0CFmThIotMAWNGo4DRCL197j4zrOvDQ8rgtrPc4XWIuLbeUCKFpAk='

In [101]:
len(history['article'])

10

In [109]:
headers = {
    'Cookie': """SUID=2F2E40613921940A000000005C7C8A68; SUV=0054C7AB61402E2F5C7C8A6C4E9DE485; IPLOC=CN3100; UM_distinctid=16af8a3d10b1c6-00075670c610bf-e353165-1fa400-16af8a3d10c66e; ABTEST=5|1560408316|v1; JSESSIONID=aaafTsCy_KK2le_X5lqRw; PHPSESSID=38sddn82q3fr3mihpq5s9j09j7; SNUID=A3091D2C494FC612B1C5C56E4A2CA7BE; sct=7""",
    'Host': 'weixin.sogou.com',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
}

In [77]:
ua = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]


In [85]:
response = requests.get(url, headers=headers)

In [90]:
if response.status_code == 200:
    article = response.text


In [91]:
response.status_code

200

In [89]:
with open('{0}_{title}.html'.format(title, article), 'w') as fp:
    fp.write(response.text)